In [1]:
from transformers import pipeline
import torch
from transformers import AutoTokenizer, AutoModel

d:\subject\nlp\code\natural_language_processing\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Masked token

In [2]:
mask_filler = pipeline("fill-mask", model="xlm-roberta-base")
input_sentence = "Hanoi is the <mask> of Vietnam."

preds = mask_filler(input_sentence, top_k=5)

print(f"Câu gốc: {input_sentence}")
for pred in preds:
    print(f"Dự đoán: '{pred['token_str']}' với độ tin cậy: {pred['score']:.4f}")
    print(f" -> Câu hoàn chỉnh: {pred['sequence']}")

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


Câu gốc: Hanoi is the <mask> of Vietnam.
Dự đoán: 'capital' với độ tin cậy: 0.9354
 -> Câu hoàn chỉnh: Hanoi is the capital of Vietnam.
Dự đoán: 'center' với độ tin cậy: 0.0251
 -> Câu hoàn chỉnh: Hanoi is the center of Vietnam.
Dự đoán: 'heart' với độ tin cậy: 0.0109
 -> Câu hoàn chỉnh: Hanoi is the heart of Vietnam.
Dự đoán: 'centre' với độ tin cậy: 0.0032
 -> Câu hoàn chỉnh: Hanoi is the centre of Vietnam.
Dự đoán: 'city' với độ tin cậy: 0.0030
 -> Câu hoàn chỉnh: Hanoi is the city of Vietnam.


# Next token prediction

In [13]:
# 1. Tải pipeline "text-generation"
# Pipeline này sẽ tự động tải một mô hình phù hợp (thường là GPT-2)
generator = pipeline("text-generation", model="Qwen/Qwen3-0.6B")

# 2. Đoạn văn bản mồi
prompt = "The best thing about learning NLP is"

# 3. Sinh văn bản
# max_length: tổng độ dài của câu mồi và phần được sinh ra
# num_return_sequences: số lượng chuỗi kết quả muốn nhận
generated_texts = generator(prompt, max_new_tokens=100, num_return_sequences=1)

# 4. In kết quả
print(f"Câu mồi: '{prompt}'")
for text in generated_texts:
    print("Văn bản được sinh ra:")
    print(text['generated_text'])

Device set to use cpu


Câu mồi: 'The best thing about learning NLP is'
Văn bản được sinh ra:
The best thing about learning NLP is that it offers a way to understand and interpret how people communicate, including nonverbal cues, emotions, and language. It's not just about the language itself, but how people use it. The field is rapidly growing, and it's becoming more and more important in our daily lives. It's a powerful tool for professionals who work with language, including those in education, business, and healthcare.

NLP can be used in multiple ways, including in the workplace, in education, and in healthcare


# Sentence Representation

In [4]:
# 1. Chọn một mô hình BERT
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# 2. Câu đầu vào
sentences = [
    "This is a sample sentence.",
    "Attentions weights after the attention softmax, used to compute the weighted average in the self-attention heads."
             
]

# 3. Tokenize câu
# padding=True: đệm các câu ngắn hơn để có cùng độ dài
# truncation=True: cắt các câu dài hơn
# return_tensors='pt': trả về kết quả dưới dạng PyTorch tensors
inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
print(inputs)

# 4. Đưa qua mô hình để lấy hidden states
# torch.no_grad() để không tính toán gradient, tiết kiệm bộ nhớ
with torch.no_grad():
    outputs = model(**inputs)

# outputs.last_hidden_state chứa vector đầu ra của tất cả các token
last_hidden_state = outputs.last_hidden_state

# shape: (batch_size, sequence_length, hidden_size)

# 5. Thực hiện Mean Pooling
# Để tính trung bình chính xác, chúng ta cần bỏ qua các token đệm (padding tokens)
attention_mask = inputs['attention_mask']
print("Last hiddent state:", last_hidden_state.shape)
print("Attention mask shape:", attention_mask.shape)

mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
sum_embeddings = torch.sum(last_hidden_state * mask_expanded, 1)
sum_mask = torch.clamp(mask_expanded.sum(1), min=1e-9)
sentence_embedding = sum_embeddings / sum_mask

# 6. In kết quả
print("Vector biểu diễn của câu:")
print(sentence_embedding)
print("\nKích thước của vector:", sentence_embedding.shape)

{'input_ids': tensor([[  101,  2023,  2003,  1037,  7099,  6251,  1012,   102,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0],
        [  101,  3086,  2015, 15871,  2044,  1996,  3086,  3730, 17848,  1010,
          2109,  2000, 24134,  1996, 18215,  2779,  1999,  1996,  2969,  1011,
          3086,  4641,  1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
Last hiddent state: torch.Size([2, 24, 768])
Attention mask shape: torch.Size([2, 24])
Vector biểu diễn của câu:
tensor([[-0.0639, -0.4284, -0.0668,  ..., -0.1753, -0.1239,  0.3197],
        [-0.1905, -0.3300,  0.3282,  ...,